In [2]:
import numbers
import re
import pickle
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files

In [3]:
reviews = load_files('review_polarity/txt_sentoken/')

In [4]:
reviews["data"]
reviews.target


array([0, 1, 1, ..., 1, 0, 0])

In [5]:
x,y = reviews.data,reviews.target

In [6]:
len(y)
#x

2000

In [7]:
with open('x.pickle','wb') as f:
    pickle.dump(x,f)

In [8]:
with open('y.pickle','wb') as f:
    pickle.dump(y,f)

In [9]:
with open('x.pickle','rb') as f:
    x = pickle.load(f)

In [10]:
with open('y.pickle','rb') as f:
    y = pickle.load(f)
y


array([0, 1, 1, ..., 1, 0, 0])

In [11]:
corpus = []
for i in range(0,len(x)):
    review = re.sub(r'\W',' ',str(x[i]))
    review = review.lower()
    review = re.sub(r'\s+[a-z]\s+',' ',review)
    review = re.sub(r'^[a-z]\s+',' ',review)
    review = re.sub(r'\s+',' ',review)
    corpus.append(review)
#corpus

In [12]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(max_features=2000,min_df=0,max_df=0.6,stop_words=stopwords.words('english'))

In [13]:
# x = vectorizer.fit_transform(corpus).toarray()


In [14]:
# from sklearn.feature_extraction.text import TfidfTransformer
# transformer = TfidfTransformer()
# x = transformer.fit_transform(x).toarray()

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1500,min_df=3,max_df=0.6,stop_words=stopwords.words('english'))
x = vectorizer.fit_transform(corpus).toarray()



In [15]:
from sklearn.model_selection import train_test_split
text_train,text_test,sent_train,sent_test = train_test_split(x,y,test_size=0.2,random_state=0)
sent_test

array([0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,

In [16]:
#train our model
from sklearn.linear_model import LogisticRegression
#from sklearn.neural_network import multilayer_perceptron
classifier = LogisticRegression()
classifier.fit(text_train,sent_train)

/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
#prediction
send_pred = classifier.predict(text_test)

In [18]:
#print(sent_test)
#send_pred
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(sent_test,send_pred)
cm
#168 result were actually -ve and our model did good
#21 were actually -ve but our model predict positive
#40 were actually positive our our model classified -ve
#171 were actually positive and our model also said +ve

array([[169,  39],
       [ 28, 164]])

In [19]:
#accuracy
(cm[0][0]+cm[1][1])/(cm[0][1]+cm[1][0]+cm[0][0]+cm[1][1])

0.8325

In [20]:
with open('clasifier.pickle','wb') as f:
    pickle.dump(classifier,f)


In [21]:
with open('tfidfmodel.pickle','wb') as f:
    pickle.dump(vectorizer,f)

In [22]:
with open ('clasifier.pickle','rb') as f:
    clf = pickle.load(f)

In [327]:
with open ('tfidfmodel.pickle','rb') as f:
    tfidf = pickle.load(f)

In [330]:
sample = ["congratulation"]
sample = tfidf.transform(sample).toarray()

In [332]:
print(classifier.predict(sample))

[1]
